In [1]:
import pandas as pd
import math
import numpy as np
import copy
import warnings
import gc
from sklearn import preprocessing
warnings.filterwarnings('ignore')

In [2]:
chunksize = 10000

user_basic_info = pd.read_csv('user_basic_info.csv', names=['UID', 'gender', 'city',
                                                            'prodName', 'ramCapacity',
                                                            'ramLeftRation', 'romCapacity',
                                                            'romLeftRation', 'color',
                                                            'fontSize', 'ct', 'carrier', 'os'])
# print(user_basic_info.head(5))

user_behavior_info = pd.read_csv('user_behavior_info.csv', names=['UID', 'bootTimes', 'AFTimes',
                                                                 'BFTimes', 'CFTimes', 'DFTimes',
                                                                 'EFTimes', 'FFTimes', 'GFSum'])
# print(user_behavior_info.head(5))

       UID  gender    city prodName  ramCapacity  ramLeftRation  romCapacity  \
0  1000001       0  c00145   p00169          3.0           0.43         32.0   
1  1000002       1   c0043    p0022          NaN           0.26          NaN   
2  1000004       1  c00281    p0013          3.0           0.36         32.0   
3  1000006       0  c00359    p0045          2.0           0.19         16.0   
4  1000007       1    c006    p0097          6.0           0.20        137.0   

   romLeftRation color  fontSize       ct        carrier   os  
0           0.46   皓月银      1.15       4g   China_Mobile  8.0  
1           0.59   渐变黑      1.00     wifi   China_Mobile  9.0  
2           0.09   幻海蓝      1.00  4g#wifi   China_Mobile  8.0  
3           0.42    灰色       NaN      NaN   China_Mobile  6.0  
4           0.50   宝石蓝      1.00  4g#wifi  China_Telecom  9.0  
       UID  bootTimes  AFTimes  BFTimes  CFTimes  DFTimes  EFTimes  FFTimes  \
0  1000001        108      0.0     0.00     1.00     0.0

In [3]:
user_behavior_info = user_behavior_info.mask(user_behavior_info==0).fillna(user_behavior_info.mean())
# print(user_behavior_info.head(5))

       UID   bootTimes   AFTimes   BFTimes  CFTimes   DFTimes  EFTimes  \
0  1000001  108.000000  0.636205  0.124205   1.0000  0.070000  2.06951   
1  1000002   14.000000  0.636205  0.170000   4.9300  1.230000  3.90000   
2  1000003   13.000000  1.000000  0.124205   7.7300  3.000000  1.70000   
3  1000004   57.000000  0.636205  0.030000   1.3700  0.630000  2.06951   
4  1000005   20.403632  0.636205  0.124205   1.6007  2.279703  2.06951   

   FFTimes         GFSum  
0  0.23407   3319.000000  
1  1.00000    245.000000  
2  0.23407   5987.000000  
3  0.23407   7460.000000  
4  0.23407  11945.093461  


In [4]:
user_basic_info = user_basic_info.set_index('UID')
basic_nan = user_basic_info.apply(lambda col: sum(col.isnull())/col.size)
user_basic_info = user_basic_info.fillna(user_basic_info.mode().iloc[0])
user_basic_info = user_basic_info.drop(columns=['city', 'prodName'])
# print(user_basic_info.head(5))

         gender  ramCapacity  ramLeftRation  romCapacity  romLeftRation color  \
UID                                                                             
1000001       0          3.0           0.43         32.0           0.46   皓月银   
1000002       1          6.0           0.26         64.0           0.59   渐变黑   
1000004       1          3.0           0.36         32.0           0.09   幻海蓝   
1000006       0          2.0           0.19         16.0           0.42    灰色   
1000007       1          6.0           0.20        137.0           0.50   宝石蓝   

         fontSize       ct        carrier   os  
UID                                             
1000001      1.15       4g   China_Mobile  8.0  
1000002      1.00     wifi   China_Mobile  9.0  
1000004      1.00  4g#wifi   China_Mobile  8.0  
1000006      1.00  4g#wifi   China_Mobile  6.0  
1000007      1.00  4g#wifi  China_Telecom  9.0  


In [5]:
user_actived_info = pd.read_csv('actived.csv', header=1, names=['UID', '主题个性', '休闲益智',
                                                      '体育竞速', '便捷生活', '儿童',
                                                      '出行导航', '动作射击', '商务',
                                                      '学习办公', '实用工具', '影音娱乐',
                                                      '拍摄美化', '教育', '新闻阅读', '旅游住宿',
                                                      '棋牌桌游', '汽车', '益智棋牌', '社交通讯',
                                                      '经营策略', '网络游戏', '美食', '表盘个性',
                                                      '角色扮演', '购物比价', '运动健康', '金融理财'])
# print(user_actived_info.head(5))

       UID  主题个性  休闲益智  体育竞速  便捷生活  儿童  出行导航  动作射击  商务  学习办公  ...  益智棋牌  社交通讯  \
0  1000010     0     0     0     1   0     6     0   0     0  ...   0.0     4   
1  1000011     0     0     0     0   0     0     1   0     0  ...   0.0     3   
2  1000013     0     0     0     1   0     4     0   0     0  ...   0.0     5   
3  1000014     0     0     0     1   0     1     0   0     0  ...   0.0     6   
4  1000016     0     0     0     0   0     0     0   0     0  ...   0.0     1   

   经营策略  网络游戏  美食  表盘个性  角色扮演  购物比价  运动健康  金融理财  
0     0     0   0   NaN     0     7     2     3  
1     1     0   0   NaN     1     3     0     0  
2     0     0   0   NaN     0     7     2     5  
3     0     0   0   NaN     0     6     0     4  
4     0     0   0   NaN     0     1     0     9  

[5 rows x 28 columns]


In [6]:
user_actived_info = user_actived_info.set_index('UID')
actived_nan = user_actived_info.apply(lambda col: sum(col.isnull())/col.size)
actived_sum = user_actived_info.sum(axis=0)
user_actived_info = user_actived_info.drop(columns=['益智棋牌', '表盘个性'])
# print(user_actived_info.head(5))

         主题个性  休闲益智  体育竞速  便捷生活  儿童  出行导航  动作射击  商务  学习办公  实用工具  ...  棋牌桌游  \
UID                                                              ...         
1000010     0     0     0     1   0     6     0   0     0     5  ...     0   
1000011     0     0     0     0   0     0     1   0     0     2  ...     0   
1000013     0     0     0     1   0     4     0   0     0     6  ...     0   
1000014     0     0     0     1   0     1     0   0     0    14  ...     0   
1000016     0     0     0     0   0     0     0   0     0     3  ...     0   

         汽车  社交通讯  经营策略  网络游戏  美食  角色扮演  购物比价  运动健康  金融理财  
UID                                                        
1000010   0     4     0     0   0     0     7     2     3  
1000011   0     3     1     0   0     1     3     0     0  
1000013   0     5     0     0   0     0     7     2     5  
1000014   1     6     0     0   0     0     6     0     4  
1000016   0     1     0     0   0     0     1     0     9  

[5 rows x 25 columns]


In [7]:
user_usage_info = pd.read_csv('max_usage.csv', header=1, names=['UID', 'max_duration',
                                                               'max_times', 'max_freuency'])
# print(user_usage_info.head(5))

       UID max_duration max_times max_freuency
0  1000002         影音娱乐      社交通讯         购物比价
1  1000003         社交通讯      社交通讯         实用工具
2  1000004         影音娱乐      社交通讯         购物比价
3  1000007         影音娱乐      购物比价         购物比价
4  1000008         社交通讯      社交通讯         社交通讯


In [8]:
user = user_basic_info.merge(user_behavior_info, on='UID', how='left')
user = user.merge(user_usage_info, on='UID', how='left')
user = user.merge(user_actived_info, on='UID', how='left')
user = user.dropna()
# print(user.head(5))

       UID  gender  ramCapacity  ramLeftRation  romCapacity  romLeftRation  \
1  1000002       1          6.0           0.26         64.0           0.59   
2  1000004       1          3.0           0.36         32.0           0.09   
4  1000007       1          6.0           0.20        137.0           0.50   
8  1000012       0          4.0           0.31         64.0           0.39   
9  1000015       1          3.0           0.34         32.0           0.06   

  color  fontSize       ct        carrier  ...  棋牌桌游   汽车  社交通讯  经营策略  网络游戏  \
1   渐变黑   1.00000     wifi   China_Mobile  ...   0.0  2.0   6.0   2.0   0.0   
2   幻海蓝   1.00000  4g#wifi   China_Mobile  ...   0.0  0.0   5.0   0.0   0.0   
4   宝石蓝   1.00000  4g#wifi  China_Telecom  ...   0.0  1.0   9.0   0.0   0.0   
8   琥珀金   1.30001     wifi   China_Mobile  ...   0.0  0.0   4.0   0.0   0.0   
9   皓月银   1.30000     wifi  China_Telecom  ...   0.0  0.0   2.0   0.0   0.0   

    美食  角色扮演  购物比价  运动健康  金融理财  
1  3.0   0.0   6.0   0.

In [9]:
train = pd.read_csv('age_train.csv', names=['UID', 'age'])
trainData = train.merge(user,on='UID',how='left').dropna()
trainFeature = trainData.drop(['UID', 'age'],axis=1)
trainLabel = trainData.age

# print(trainFeature.head(5))
# print(trainLabel.head(5))

    gender  ramCapacity  ramLeftRation  romCapacity  romLeftRation color  \
4        1          3.0           0.34         32.0           0.06   皓月银   
7        1          2.0           0.34         16.0           0.06    银色   
9        0          4.0           0.31         64.0           0.20   琥珀金   
14       0          6.0           0.26         68.0           0.64   宝石蓝   
15       0          4.0           0.29         64.0           0.16   流光金   

    fontSize       ct        carrier   os  ...  棋牌桌游   汽车  社交通讯  经营策略  网络游戏  \
4       1.30     wifi  China_Telecom  8.0  ...   0.0  0.0   2.0   0.0   0.0   
7       1.00     wifi  China_Telecom  7.0  ...   0.0  0.0   3.0   0.0   0.0   
9       1.00  4g#wifi  China_Telecom  8.0  ...   0.0  2.0   8.0   0.0   0.0   
14      1.15     wifi  China_Telecom  9.0  ...   0.0  0.0   3.0   0.0   0.0   
15      1.00  4g#wifi   China_Mobile  7.0  ...   0.0  0.0   3.0   0.0   0.0   

     美食  角色扮演  购物比价 运动健康  金融理财  
4   0.0   0.0   3.0  0.0   2.0  
7 

In [10]:
trainFeature = pd.get_dummies(trainFeature)
# print(trainFeature.head(5))

    gender  ramCapacity  ramLeftRation  romCapacity  romLeftRation  fontSize  \
4        1          3.0           0.34         32.0           0.06      1.30   
7        1          2.0           0.34         16.0           0.06      1.00   
9        0          4.0           0.31         64.0           0.20      1.00   
14       0          6.0           0.26         68.0           0.64      1.15   
15       0          4.0           0.29         64.0           0.16      1.00   

     os  bootTimes   AFTimes   BFTimes  ...  max_freuency_棋牌桌游  \
4   8.0       12.0  0.636205  0.124205  ...                  0   
7   7.0        5.0  0.636205  0.124205  ...                  0   
9   8.0       97.0  0.636205  0.170000  ...                  0   
14  9.0        5.0  1.000000  0.100000  ...                  0   
15  7.0       42.0  0.636205  0.130000  ...                  0   

    max_freuency_汽车  max_freuency_社交通讯  max_freuency_经营策略  max_freuency_网络游戏  \
4                 0                  1    

In [17]:
trainLabel = trainLabel.apply(lambda x: x-1)
# print(trainLabel.head(5))

4     4.0
7     1.0
9     3.0
14    1.0
15    2.0
Name: age, dtype: float64


In [20]:
import xgboost as xgb
import operator

params = {
    'booster':'gbtree',
    'objective':'multi:softmax',
    'num_class':6,
    'stratified':True,
    'max_depth':5,
    'min_child_weight':1,
    'gamma':3,
    'subsample':0.8,#0.7
    'colsample_bytree':0.6, 
    'lambda':3, 
    'eta':0.05,
    'seed':20,
    'silent':1,
    'eval_metric':'auc'
}

In [18]:
dtrain = xgb.DMatrix(trainFeature, label=trainLabel)
gbdt = xgb.train(params, dtrain, 1000)

In [22]:
importance = gbdt.get_fscore()
importance = sorted(importance.items(), key=operator.itemgetter(1))
# print(importance)

[(u'max_times_\u793e\u4ea4\u901a\u8baf', 1), (u'max_duration_\u7ecf\u8425\u7b56\u7565', 1), (u'max_freuency_\u793e\u4ea4\u901a\u8baf', 1), (u'\u5f71\u97f3\u5a31\u4e50', 1), (u'bootTimes', 1), (u'max_duration_\u5f71\u97f3\u5a31\u4e50', 1), (u'max_duration_\u68cb\u724c\u684c\u6e38', 1), (u'max_freuency_\u5b9e\u7528\u5de5\u5177', 1), (u'max_freuency_\u6559\u80b2', 1), (u'ct_wifi', 1), (u'color_\u6469\u5361\u91d1', 1), (u'AFTimes', 2), (u'\u6c7d\u8f66', 2), (u'ramCapacity', 2), (u'\u65c5\u6e38\u4f4f\u5bbf', 3), (u'\u4f53\u80b2\u7ade\u901f', 4), (u'\u7ecf\u8425\u7b56\u7565', 5), (u'DFTimes', 5), (u'max_freuency_\u5f71\u97f3\u5a31\u4e50', 6), (u'os', 6), (u'\u5546\u52a1', 7), (u'\u5b9e\u7528\u5de5\u5177', 8), (u'fontSize', 8), (u'\u793e\u4ea4\u901a\u8baf', 8), (u'\u62cd\u6444\u7f8e\u5316', 10), (u'\u51fa\u884c\u5bfc\u822a', 11), (u'\u6559\u80b2', 11), (u'\u91d1\u878d\u7406\u8d22', 12), (u'GFSum', 12), (u'\u52a8\u4f5c\u5c04\u51fb', 13), (u'\u513f\u7ae5', 15), (u'EFTimes', 16)]


In [ ]:
test = pd.read_csv('age_test.csv', names=['UID', 'age'])
testData = test.merge(user,on='UID',how='left').dropna()
testFeature = testData.drop(['UID', 'age'],axis=1)
dtest = xgb.DMatrix(testFeature.values)
predict = gbdt.predict(dtest)
result = pd.DataFrame()
result['UID'] = test['UID']
result['Tag'] = predict
print(result.head(5))

In [ ]:
result.to_csv('result.csv',index=0)